In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup

from sibulator import *

In [2]:
known_sample_path = 'AfAm Noncontributor Database.csv'
allele_freq_data = 'nist'
subpop = 'AfAm'
num_siblings = 1000
dest_path = 'test.csv'

# generate samples
known_sample = load_known_sample(known_sample_path)
sibling_samples = generate_sibling_samples(known_sample, allele_freq_data, subpop, num_siblings)

In [18]:
output = {}
col_rename = {}
for i, sample in enumerate(sibling_samples):
    for k, v in sample.items():
        output[k] = output.get(k, []) + [float(v[0])] # bit hacky that some are single item lists
        output[k + '.1'] = output.get(k + '.1', []) + [float(v[1])] # bit hacky that some are single item lists
        if i == 0:
            col_rename[k + '.1'] = k
output = pd.DataFrame(output)
output = output.rename(col_rename, axis=1)
output.head()

,D3S1358,D3S1358,D1S1656,D1S1656,D2S441,D2S441,D10S1248,D10S1248,D13S317,D13S317,...,D8S1179,D8S1179,D12S391,D12S391,D19S433,D19S433,FGA,FGA,D22S1045,D22S1045
0,12.2,30.3,17.3,15.3,12.3,13.0,22.0,13.0,3.2,8.0,...,32.2,18.0,25.2,18.3,6.3,10.0,22.0,24.0,34.2,15.0
1,15.0,3.2,19.4,15.3,15.0,13.0,21.0,39.0,19.4,8.0,...,10.3,30.2,30.3,8.0,33.1,34.0,19.1,3.2,14.0,16.0
2,15.4,15.2,24.3,15.3,17.1,29.2,5.0,13.0,13.3,8.0,...,9.0,11.0,39.0,18.3,14.2,19.0,8.0,14.3,18.0,12.3
3,19.3,15.2,17.3,15.3,23.0,13.0,13.2,19.0,30.2,8.0,...,19.4,11.0,21.0,39.0,27.0,10.0,18.2,24.0,17.3,16.0
4,17.2,15.2,17.3,15.3,12.3,13.0,19.4,13.0,13.4,8.0,...,18.1,11.0,16.2,18.3,15.2,14.2,18.3,24.0,23.2,16.0


In [17]:
output.to_csv(dest_path)

In [12]:
data = pd.read_csv(known_sample_path).iloc[0] # assume first row is desired known profile
# print(data)
known_sample = {}
for loc in data.drop(['CaseNumber', 'Sample']).index:
    print(loc)
    allele = data[loc]
    key = loc.split('.')[0]
    known_sample[key] = known_sample.get(key, []) + [allele]

D3S1358
D3S1358.1
D1S1656
D1S1656.1
D2S441
D2S441.1
D10S1248
D10S1248.1
D13S317
D13S317.1
Penta_E
Penta_E.1
D16S539
D16S539.1
D18S51
D18S51.1
D2S1338
D2S1338.1
CSF1PO
CSF1PO.1
Penta_D
Penta_D.1
TH01
TH01.1
vWA
vWA.1
D21S11
D21S11.1
D7S820
D7S820.1
D5S818
D5S818.1
TPOX
TPOX.1
D8S1179
D8S1179.1
D12S391
D12S391.1
D19S433
D19S433.1
FGA
FGA.1
D22S1045
D22S1045.1


In [105]:
allele_freq_dict = {}
# read data in
afam_data = pd.read_csv('NIST/NIST Fusion AfAm_Amended2017.csv')
asian_data = pd.read_csv('NIST/NIST Fusion Asian_Amended2017.csv')
cauc_data = pd.read_csv('NIST/NIST Fusion Cauc_Amended2017.csv')
hisp_data = pd.read_csv('NIST/NIST Fusion Hisp_Amended2017.csv')
data = {'AfAm': afam_data, 'Asian': asian_data, 'Cauc': cauc_data, 'Hisp': hisp_data}
N = []
for df in data.values():
    df_n = df[df['Allele'] == 'N']
    N.append(int(df_n.values[0][1]))

    # drop allele from index
    df.drop(df_n.index[0], axis=0, inplace=True)
    
print(N)

# convert data to dictionaries in expected format, also combine to create a total population frequency
i = 0
total_allele_counts = {}
for subpop, df in data.items():
    for col in df.drop('Allele', axis=1).columns:
        # column is the location name ex. D3S1358
        allele_freq_dict[col] = {**allele_freq_dict.get(col, {}), **{subpop: {'alleles': df['Allele'].values, 'frequency': df[col].values}}}
        
        # update total counts of each allele
        n = N[i]
        for j, allele in enumerate(df['Allele']):
            # iterate over Series
            allele_freq = df.iloc[j][col] # allele frequency at current location
            allele_count = allele_freq * n
            temp_dict = total_allele_counts.get(col, {}) # get allele counts associated w/ location
            temp_dict[allele] = temp_dict.get(allele, 0) + allele_count  # update counts
            total_allele_counts[col] = temp_dict # replace counts
            
    i += 1

# add total population allele frequencies to data
for loc, counts in total_allele_counts.items():
    for k, v in counts.items():
        total_allele_counts[loc][k] = v / sum(N) # normalize counts by total number of profiles
    allele_freq_dict[loc]['total'] = {'alleles': list(counts.keys()), 'frequency': list(counts.values())}

[684, 194, 722, 472]


In [106]:
allele_freq_dict['D3S1358']

{'AfAm': {'alleles': array(['2.2', '3.2', '4', '4.2', '5', '6', '6.3', '7', '8', '8.1', '9',
         '9.1', '9.3', '10', '10.1', '10.2', '10.3', '11', '11.2', '11.3',
         '12', '12.2', '12.3', '13', '13.2', '13.3', '13.4', '14', '14.2',
         '14.3', '15', '15.2', '15.3', '15.4', '16', '16.2', '16.3', '17',
         '17.1', '17.2', '17.3', '18', '18.1', '18.2', '18.3', '19', '19.1',
         '19.2', '19.3', '19.4', '20', '20.1', '20.2', '20.3', '21', '21.2',
         '21.3', '22', '22.2', '22.3', '23', '23.2', '23.3', '24', '24.2',
         '24.3', '25', '25.2', '26', '26.2', '27', '27.2', '27.3', '28',
         '28.2', '28.3', '29', '29.2', '29.3', '30', '30.2', '30.3', '31',
         '31.2', '32', '32.2', '33', '33.1', '33.2', '34', '34.2', '35',
         '36', '37', '38', '39', '43.2'], dtype=object),
  'frequency': array([0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
         0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
         0.    ,

In [ ]:
allele_freq_dict

In [76]:
allele_freq = pd.read_csv('NIST/NIST Fusion AfAm_Amended2017.csv')
allele_freq.head()

,Allele,D3S1358,D1S1656,D2S441,D10S1248,D13S317,Penta_E,D16S539,D18S51,D2S1338,...,D21S11,D7S820,D5S818,TPOX,DYS391,D8S1179,D12S391,D19S433,FGA,D22S1045
0,2.2,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
1,3.2,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
2,4,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
3,4.2,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.095,0.0015,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [88]:
allele_freq['D3S1358']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
      ...  
93      0.0
94      0.0
95      0.0
96      0.0
97    684.0
Name: D3S1358, Length: 98, dtype: float64

In [ ]:
# read in distribution of alleles in given populations/subpopulations
# allele_freq = pd.read_csv(_) # should be formatted as dict based on later use

In [2]:
# Get known sample
# format to be dict mapping allele to tuple/list, read from file would be easiest
known_sample = _ 

In [ ]:
'''
Currently replacing one at each location
Move to at each location, randomly choose whether to replace 0, 1, or 2 alleles
Calibrate to population prevalence of overlap to get thresholds for 0,1,2 replacements
'''
# Generate sibling profiles
N = 1000 # number of simulated siblings
origin = '' # population of interest
simulated_samples = []
for i in range(N):
    rand_sample = {}
    for loci, allele in known_sample.items():
        # at each loci, choose to either replae 0, 1, or 2 alleles
        a1, a2 = allele
        test = np.random.random()
        if test < 0.25:
            # replace none
            new_alleles = [a1, a2]
        elif:
            # replace one
            a3 = np.random.choice(allele_freq[loci][origin]['alleles'], allele_freq[loci][origin]['frequency'])
            if np.random.random() < 0.5:
                # replace 1st
                new_alleles = [a3, a2]
            else:
                # replace second
                new_alleles = [a3, a2]
        else:
            a3, a4 = np.random.choice(allele_freq[loci][origin]['alleles'], allele_freq[loci][origin]['frequency'])
            new_alleles = [a3, a4]
        rand_sample[loci] = new_alleles
    simulated_samples.append(rand_sample)

In [2]:
# Reading the data inside the xml file to a variable under the name data
with open('STRidER_frequencies_2019-08-02.xml', 'r') as f:
    data = f.read()

# Passing the stored data inside the beautifulsoup parser, storing the returned object 
bs_data = BeautifulSoup(data, "xml")

In [55]:
# turn xml into dict for use
marker_data = bs_data.find_all('marker')
print(len(marker_data))
allele_freq_dict = {}
for md in marker_data:
    loc_name = md.find('name').text
    allele_freq_dict[loc_name] = {}
    
    alleles_list = md.find('alleles').text.split(', ')
    origins = md.find_all('origin')
    for orig in origins:
        orig_name = orig.get('name')
        frequency_dict = {}
        allele_freqs = orig.find_all('frequency')
        for allele in allele_freqs:
            al = allele.get('allele')
            freq = float(allele.text)
            frequency_dict[al] = float(allele.text)
        for al in alleles_list:
            frequency_dict[al] = frequency_dict.get(al, 0)
            
        al_list = list(frequency_dict.keys())
        freq_list = list(frequency_dict.values())
        print(sum(freq_list))
    
        allele_freq_dict[loc_name][orig_name] = {'alleles': al_list}
        allele_freq_dict[loc_name][orig_name] = {'frequency': freq_list}

41
1.0000006000000001
1.00000065
1.0000005
1.0000000000000002
1.0
0.99999893
0.99999944
0.9999998639999998
1.0000002199999998
0.9999995600000001
1.00000066
0.9999999999999998
0.9999993
0.99999981
0.99999953
0.999999568
1.00000121
0.99999922
1.0000005
0.999999107
0.999999107
0.9999990834000002
0.9999998437000002
0.9999998399999999
0.99999878
0.99999993
1.0
1.0
1.00000006
1.0000001900000002
1.000000261
1.0000006
0.9999992799999999
1.00000054
0.9999999999999999
0.9999997199999999
1.00000035
0.9999981900000001
1.000000642
0.9999998199999999
1.0000002799999999
0.9999995000000002
1.0000002669999999
1.000000267
0.999999857
0.9999991339999998
0.99999995
0.9999992399999998
0.9999994400000001
1.0
1.0
1.00000102
0.99999979
1.00000011
0.9999996800000001
1.00000114
1.00000054
0.9999999999999998
1.0
1.0000003300000002
0.99999945
0.9999993180000002
0.99999972
1.00000086
1.0
1.000000637
1.000000637
1.0000005207999998
1.0000007146
1.0000001
1.0000000999999998
0.9999991800000001
0.9999999999999999
1.0
0

In [52]:
test_marker.find_all('origin')

[<origin n="222" name="AUSTRIA"><frequency allele="1">2.25225e-3</frequency><frequency allele="16">1.03604e-1</frequency><frequency allele="19.3">1.12613e-2</frequency><frequency allele="11">8.78378e-2</frequency><frequency allele="16.3">4.95495e-2</frequency><frequency allele="20.3">2.25225e-3</frequency><frequency allele="12">1.14865e-1</frequency><frequency allele="17">4.95495e-2</frequency><frequency allele="13">5.85586e-2</frequency><frequency allele="17.3">1.53153e-1</frequency><frequency allele="14">9.68468e-2</frequency><frequency allele="13.3">2.25225e-3</frequency><frequency allele="18">4.50450e-3</frequency><frequency allele="15">1.39640e-1</frequency><frequency allele="18.3">4.27928e-2</frequency><frequency allele="15.3">7.88288e-2</frequency><frequency allele="19">2.25225e-3</frequency></origin>,
 <origin n="206" name="BELGIUM"><frequency allele="12">1.23786e-1</frequency><frequency allele="16">1.38350e-1</frequency><frequency allele="13">3.64078e-2</frequency><frequency a

In [48]:
test_marker = marker_data[0]
alleles_list = test_marker.find('alleles').text
freq_list = []
for allele in test_marker.find_all('frequency'):
    al = allele.get('allele')
    freq = float(allele.text)
    freq_list.append(freq)
    # print(al in alleles_list, al, freq)
print(sum(freq_list))
test_marker.find_all('frequency')

22.999996703099967


[<frequency allele="1">2.25225e-3</frequency>,
 <frequency allele="16">1.03604e-1</frequency>,
 <frequency allele="19.3">1.12613e-2</frequency>,
 <frequency allele="11">8.78378e-2</frequency>,
 <frequency allele="16.3">4.95495e-2</frequency>,
 <frequency allele="20.3">2.25225e-3</frequency>,
 <frequency allele="12">1.14865e-1</frequency>,
 <frequency allele="17">4.95495e-2</frequency>,
 <frequency allele="13">5.85586e-2</frequency>,
 <frequency allele="17.3">1.53153e-1</frequency>,
 <frequency allele="14">9.68468e-2</frequency>,
 <frequency allele="13.3">2.25225e-3</frequency>,
 <frequency allele="18">4.50450e-3</frequency>,
 <frequency allele="15">1.39640e-1</frequency>,
 <frequency allele="18.3">4.27928e-2</frequency>,
 <frequency allele="15.3">7.88288e-2</frequency>,
 <frequency allele="19">2.25225e-3</frequency>,
 <frequency allele="12">1.23786e-1</frequency>,
 <frequency allele="16">1.38350e-1</frequency>,
 <frequency allele="13">3.64078e-2</frequency>,
 <frequency allele="16.3">5

In [ ]:
# Finding all instances of tag 
# `unique`
b_unique = Bs_data.find_all('unique')
  
print(b_unique)
  
# Using find() to extract attributes 
# of the first instance of the tag
b_name = Bs_data.find('child', {'name':'Frank'})
  
print(b_name)
  
# Extracting the data stored in a
# specific attribute of the 
# `child` tag
value = b_name.get('test')